In [ ]:
!pip install --upgrade Flask web3 Flask-Cors
from google.colab import files
uploaded = files.upload()

Saving abi.json to abi (1).json


In [ ]:
import json
from web3 import Web3
# from web3.middleware import geth_poa_middleware # Tạm thời loại bỏ dòng này để khắc phục lỗi Attribute Error
import os

# --- Cấu hình của bạn (Đảm bảo an toàn cho khóa riêng trong môi trường thực tế!) ---
# Địa chỉ hợp đồng đã triển khai của bạn
CONTRACT_ADDRESS = "0x297AAd040B970B09a31573D8bCe7Fe3215B2cc33"
# ABI của hợp đồng (Application Binary Interface)
CONTRACT_ABI = json.loads("""
[{"inputs":[{"internalType":"uint256","name":"_id","type":"uint256"},{"internalType":"string","name":"_newContent","type":"string"}],"name":"editMessage","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_id","type":"uint256"}],"name":"markAsDeleted","outputs":[],"stateMutability":"nonpayable","type":"function"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"messageId","type":"uint256"},{"indexed":true,"internalType":"address","name":"deleter","type":"address"},{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"}],"name":"MessageDeleted","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"messageId","type":"uint256"},{"indexed":true,"internalType":"address","name":"editor","type":"address"},{"indexed":false,"internalType":"string","name":"newContent","type":"string"},{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"}],"name":"MessageEdited","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"messageId","type":"uint256"},{"indexed":true,"internalType":"address","name":"liker","type":"address"},{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"}],"name":"MessageLiked","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"messageId","type":"uint256"},{"indexed":true,"internalType":"address","name":"unliker","type":"address"},{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"}],"name":"MessageUnliked","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"messageId","type":"uint256"},{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":false,"internalType":"string","name":"content","type":"string"},{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"}],"name":"NewMessage","type":"event"},{"inputs":[{"internalType":"string","name":"_content","type":"string"}],"name":"postMessage","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_id","type":"uint256"}],"name":"toggleLike","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_id","type":"uint256"},{"internalType":"address","name":"_user","type":"address"}],"name":"getLikeStatus","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"_index","type":"uint256"}],"name":"getMessage","outputs":[{"components":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"string","name":"content","type":"string"},{"internalType":"address","name":"sender","type":"address"},{"internalType":"uint256","name":"timestamp","type":"uint256"},{"internalType":"uint256","name":"lastEditedTimestamp","type":"uint256"},{"internalType":"bool","name":"isDeleted","type":"bool"},{"internalType":"uint256","name":"likesCount","type":"uint256"}],"internalType":"struct SimpleMessage.Message","name":"","type":"tuple"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"_sender","type":"address"}],"name":"getMessagesBySender","outputs":[{"components":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"string","name":"content","type":"string"},{"internalType":"address","name":"sender","type":"address"},{"internalType":"uint256","name":"timestamp","type":"uint256"},{"internalType":"uint256","name":"lastEditedTimestamp","type":"uint256"},{"internalType":"bool","name":"isDeleted","type":"bool"},{"internalType":"uint256","name":"likesCount","type":"uint256"}],"internalType":"struct SimpleMessage.Message[]","name":"","type":"tuple[]"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getTotalMessages","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"messages","outputs":[{"internalType":"uint256","name":"id","type":"uint256"},{"internalType":"string","name":"content","type":"string"},{"internalType":"address","name":"sender","type":"address"},{"internalType":"uint256","name":"timestamp","type":"uint256"},{"internalType":"uint256","name":"lastEditedTimestamp","type":"uint256"},{"internalType":"bool","name":"isDeleted","type":"bool"},{"internalType":"uint256","name":"likesCount","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"nextMessageId","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]""")
# URL của nhà cung cấp Web3 (ví dụ: Infura, Alchemy)
WEB3_PROVIDER_URL = "https://sepolia.infura.io/v3/bed0f0a403ff4929a13d7c3a3d58a24d"
# Khóa riêng của tài khoản sẽ gửi giao dịch (KHÔNG BAO GIỜ ĐỂ LỘ TRONG MÔI TRƯỜNG SẢN XUẤT!)
# Nên sử dụng biến môi trường: PRIVATE_KEY = os.getenv("PRIVATE_KEY")
PRIVATE_KEY = ""

# --- Khởi tạo Web3 và Hợp đồng ---
w3 = Web3(Web3.HTTPProvider(WEB3_PROVIDER_URL))

# # Thêm middleware cho các mạng Proof-of-Authority (như Sepolia)
# # Tạm thời loại bỏ dòng này để khắc phục lỗi Attribute Error
# w3.middleware_onion.inject(geth_poa_middleware, layer=0)

# Kiểm tra kết nối
if w3.is_connected():
    print("Đã kết nối thành công với mạng Ethereum!")
else:
    print("Không thể kết nối với mạng Ethereum. Vui lòng kiểm tra WEB3_PROVIDER_URL.")
    exit()

# Lấy tài khoản gửi giao dịch từ khóa riêng
account = w3.eth.account.from_key(PRIVATE_KEY)
print(f"Tài khoản gửi giao dịch: {account.address}")

# Tải hợp đồng
contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=CONTRACT_ABI)

# --- Hàm trợ giúp để gửi giao dịch ---
def send_transaction(function_call):
    """
    Ký và gửi một giao dịch đến hợp đồng thông minh.
    Args:
        function_call: Cuộc gọi hàm của hợp đồng (ví dụ: contract.functions.postMessage("Hello"))
    Returns:
        hash của giao dịch đã gửi.
    """
    try:
        # Lấy nonce cho tài khoản
        nonce = w3.eth.get_transaction_count(account.address)

        # Xây dựng giao dịch
        transaction = function_call.build_transaction({
            'chainId': w3.eth.chain_id,
            'gasPrice': w3.eth.gas_price,
            'from': account.address,
            'nonce': nonce,
        })

        # Ký giao dịch
        signed_txn = w3.eth.account.sign_transaction(transaction, private_key=PRIVATE_KEY)

        # Gửi giao dịch
        tx_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
        print(f"Giao dịch đã gửi. Tx Hash: {tx_hash.hex()}")

        # Đợi giao dịch được xác nhận
        receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
        print(f"Giao dịch đã được xác nhận. Trạng thái: {receipt.status}")
        if receipt.status == 1:
            print("Giao dịch thành công!")
        else:
            print("Giao dịch thất bại!")
        return tx_hash.hex()
    except Exception as e:
        print(f"Lỗi khi gửi giao dịch: {e}")
        return None

# --- Chức năng tương tác với hợp đồng ---

def post_new_message(content: str):
    """Gửi một tin nhắn mới."""
    print(f"\nĐang gửi tin nhắn mới: '{content}'...")
    function_call = contract.functions.postMessage(content)
    return send_transaction(function_call)

def edit_existing_message(message_id: int, new_content: str):
    """Chỉnh sửa nội dung của một tin nhắn."""
    print(f"\nĐang chỉnh sửa tin nhắn ID {message_id} thành: '{new_content}'...")
    function_call = contract.functions.editMessage(message_id, new_content)
    return send_transaction(function_call)

def mark_message_as_deleted(message_id: int):
    """Đánh dấu một tin nhắn là đã xóa."""
    print(f"\nĐang đánh dấu tin nhắn ID {message_id} là đã xóa...")
    function_call = contract.functions.markAsDeleted(message_id)
    return send_transaction(function_call)

def toggle_message_like(message_id: int):
    """Thích hoặc bỏ thích một tin nhắn."""
    print(f"\nĐang thích/bỏ thích tin nhắn ID {message_id}...")
    function_call = contract.functions.toggleLike(message_id)
    return send_transaction(function_call)

def get_message_details(index: int):
    """Lấy thông tin chi tiết của một tin nhắn theo chỉ mục."""
    print(f"\nĐang lấy chi tiết tin nhắn tại chỉ mục {index}...")
    try:
        message = contract.functions.getMessage(index).call()
        # Chuyển đổi timestamp sang định dạng dễ đọc
        import datetime
        # message[3] là timestamp, message[4] là lastEditedTimestamp
        timestamp_dt = datetime.datetime.fromtimestamp(message[3])
        edited_timestamp_dt = datetime.datetime.fromtimestamp(message[4]) if message[4] > 0 else "N/A"

        print(f"  ID: {message[0]}")
        print(f"  Nội dung: {message[1]}")
        print(f"  Người gửi: {message[2]}")
        print(f"  Thời gian gửi: {timestamp_dt}")
        print(f"  Thời gian chỉnh sửa cuối cùng: {edited_timestamp_dt}")
        print(f"  Đã xóa: {message[5]}")
        print(f"  Lượt thích: {message[6]}")
        return message
    except Exception as e:
        print(f"Lỗi khi lấy tin nhắn: {e}")
        return None

def get_total_messages():
    """Lấy tổng số tin nhắn."""
    print("\nĐang lấy tổng số tin nhắn...")
    total_messages = contract.functions.getTotalMessages().call()
    print(f"Tổng số tin nhắn: {total_messages}")
    return total_messages

def get_messages_by_sender(sender_address: str):
    """Lấy tất cả tin nhắn của một người gửi cụ thể."""
    print(f"\nĐang lấy tin nhắn của người gửi: {sender_address}...")
    try:
        messages = contract.functions.getMessagesBySender(sender_address).call()
        if not messages:
            print(f"Không tìm thấy tin nhắn nào từ {sender_address}.")
            return []
        for msg in messages:
            import datetime
            # msg[3] là timestamp, msg[4] là lastEditedTimestamp
            timestamp_dt = datetime.datetime.fromtimestamp(msg[3])
            edited_timestamp_dt = datetime.datetime.fromtimestamp(msg[4]) if msg[4] > 0 else "N/A"
            print(f"  --- Tin nhắn ID: {msg[0]} ---")
            print(f"    Nội dung: {msg[1]}")
            print(f"    Người gửi: {msg[2]}")
            print(f"    Thời gian gửi: {timestamp_dt}")
            print(f"    Thời gian chỉnh sửa cuối cùng: {edited_timestamp_dt}")
            print(f"    Đã xóa: {msg[5]}")
            print(f"    Lượt thích: {msg[6]}")
        return messages
    except Exception as e:
        print(f"Lỗi khi lấy tin nhắn theo người gửi: {e}")
        return None

def get_user_like_status(message_id: int, user_address: str):
    """Kiểm tra trạng thái thích của người dùng đối với một tin nhắn."""
    print(f"\nĐang kiểm tra trạng thái thích của {user_address} cho tin nhắn ID {message_id}...")
    try:
        has_liked = contract.functions.getLikeStatus(message_id, user_address).call()
        print(f"  Người dùng {user_address} đã thích tin nhắn ID {message_id}: {has_liked}")
        return has_liked
    except Exception as e:
        print(f"Lỗi khi kiểm tra trạng thái thích: {e}")
        return None

# --- Ví dụ sử dụng ---
if __name__ == "__main__":
    # 1. Gửi một tin nhắn mới
    post_new_message("Xin chào từ backend Python!")

    # Chờ một chút để giao dịch được xử lý trên blockchain
    import time
    time.sleep(10) # Đợi 10 giây để đảm bảo giao dịch được đào

    # 2. Lấy tổng số tin nhắn
    total = get_total_messages()

    # 3. Lấy chi tiết tin nhắn cuối cùng (nếu có)
    if total > 0:
        last_message_id = total - 1 # ID tin nhắn cuối cùng thường là total - 1
        get_message_details(last_message_id)

        # 4. Chỉnh sửa tin nhắn cuối cùng
        # Đảm bảo tài khoản gửi giao dịch là người gửi tin nhắn này để chỉnh sửa thành công
        # edit_existing_message(last_message_id, "Nội dung đã được chỉnh sửa!")
        # time.sleep(10)
        # get_message_details(last_message_id)

        # 5. Thích tin nhắn cuối cùng
        # toggle_message_like(last_message_id)
        # time.sleep(10)
        # get_message_details(last_message_id)
        # get_user_like_status(last_message_id, account.address) # Kiểm tra xem tài khoản đã thích chưa

        # 6. Đánh dấu tin nhắn cuối cùng là đã xóa (xóa mềm)
        # mark_message_as_deleted(last_message_id)
        # time.sleep(10)
        # get_message_details(last_message_id)

    # 7. Lấy tất cả tin nhắn của tài khoản hiện tại
    get_messages_by_sender(account.address)


Đã kết nối thành công với mạng Ethereum!
Tài khoản gửi giao dịch: 0xa2e9A589710c0aDf6B834a0df1f2572B47A253f5

Đang gửi tin nhắn mới: 'Xin chào từ backend Python!'...
Lỗi khi gửi giao dịch: 'SignedTransaction' object has no attribute 'rawTransaction'

Đang lấy tổng số tin nhắn...
Tổng số tin nhắn: 3

Đang lấy chi tiết tin nhắn tại chỉ mục 2...
  ID: 2
  Nội dung: toi la Thanh Tam
  Người gửi: 0xa2e9A589710c0aDf6B834a0df1f2572B47A253f5
  Thời gian gửi: 2025-05-23 06:52:12
  Thời gian chỉnh sửa cuối cùng: 2025-05-23 06:52:48
  Đã xóa: False
  Lượt thích: 0

Đang lấy tin nhắn của người gửi: 0xa2e9A589710c0aDf6B834a0df1f2572B47A253f5...
  --- Tin nhắn ID: 0 ---
    Nội dung: xin chao viet nam
    Người gửi: 0xa2e9A589710c0aDf6B834a0df1f2572B47A253f5
    Thời gian gửi: 2025-05-23 06:49:24
    Thời gian chỉnh sửa cuối cùng: N/A
    Đã xóa: False
    Lượt thích: 0
  --- Tin nhắn ID: 1 ---
    Nội dung: toi la Quang Duy
    Người gửi: 0xa2e9A589710c0aDf6B834a0df1f2572B47A253f5
    Thời gian gửi